In [1]:
import geopandas as gpd

In [4]:
amenities_file = '/app/data/amenities'

amn = gpd.read_file(amenities_file)
amn['name'].fillna('', inplace=True)

idna = amn['id'].isna()
amn.loc[idna, 'id'] = ('2024' + amn[idna].index.astype(str)).astype(int)

futr = amn['name'].str.contains('futur', case=False)
amn.loc[~futr, 'scenario'] = 'actual'
amn.loc[ futr, 'scenario'] = 'future'

amn_cols = ['id', 'Category', 'scenario', 'geometry']
amn = amn[amn_cols]
# amn
# amn[amn['Category']=='Cultura']

In [5]:
amn = gpd.GeoDataFrame(data=amn.drop(columns=['geometry']), geometry=amn['geometry'])

In [6]:
amn['id'] = amn['id'].astype(str)

In [7]:
amn.loc[amn['scenario']=='actual', ['id', 'Category', 'geometry']].to_parquet('/app/assets/amenities/actual.parquet')
amn.loc[:, ['id', 'Category', 'geometry']].to_parquet('/app/assets/amenities/future.parquet')

In [8]:
amn_costanera = gpd.read_file('/app/data/amenities_costanera_v2')

In [9]:
amn_costanera['scenario'] = 'actual'

In [10]:
amn_costanera = amn_costanera[['id', 'Category', 'scenario', 'geometry']]
amn_costanera.shape

(4573, 4)

In [11]:
amn_costanera.drop_duplicates(subset=['Category', 'geometry'], inplace=True)

In [13]:
amn_costanera = amn_costanera.to_crs(amn.crs)

In [15]:
from shapely.geometry import Point

# Suponiendo que 'gdf' es tu GeoDataFrame
# Itera sobre las geometrías y crea nuevos puntos 2D
amn['geometry'] = amn['geometry'].apply(lambda geom: Point(geom.x, geom.y))

In [16]:
import pandas as pd
amn_export = pd.concat([amn, amn_costanera])
amn_export.drop_duplicates(subset=['Category', 'geometry'], inplace=True)
amn_export.drop_duplicates(subset=['id'], inplace=True)

In [17]:
amn_export = gpd.GeoDataFrame(
    data=amn_export.drop(columns=['geometry']),
    geometry=amn_export['geometry'],
)

In [18]:
export_cols = ['id', 'Category', 'geometry']
amn_export.loc[amn_export['scenario']=='actual', export_cols].to_parquet('/app/assets/amenities/actual.parquet')
amn_export[export_cols].to_parquet('/app/assets/amenities/future.parquet')

In [19]:
amn_export.loc[amn_export['scenario']=='actual', export_cols]

,id,Category,geometry
0,1280585100,Entretenimiento,POINT (673882.866 5921852.428)
1,1280906720,Comida para servir,POINT (673587.739 5921888.825)
2,1280963868,Entretenimiento,POINT (673615.644 5920946.613)
3,1507046243,Servicios,POINT (673665.025 5922374.360)
4,1507046253,Servicios,POINT (673610.110 5922345.688)
...,...,...,...
4254,2023104254,Comercio,POINT (673737.232 5920608.186)
4275,2023104275,Comercio,POINT (672794.080 5923516.786)
4494,2023104494,Comercio,POINT (672786.592 5923496.482)
4521,2023104521,Comercio,POINT (672595.199 5923399.289)


In [20]:
# exportar amenities fixeadas
gdf = gpd.read_file('/app/data/amenities_Actual')

In [21]:
gdf.loc[gdf['id'].isna(), 'id'] = ('2024' + gdf[gdf['id'].isna()].index.astype(str)).astype(int)
gdf['id'] = gdf['id'].astype(str)

In [22]:
amns = {}
amns['actual'] = pd.concat([gdf, amn_export.loc[amn_export['scenario']=='actual', export_cols]]).drop_duplicates(subset=['Category', 'geometry'])

In [6]:
# gpd.read_parquet('/app/assets/amenities/future.parquet').to_file('/app/data/output/amenities_por_corregir')

In [24]:
futuro_corregido_helen = gpd.read_file('/app/data/output/amenities_por_corregir_helen')

In [25]:
futuro_corregido_helen.loc[futuro_corregido_helen['id'].isna(), 'id'] = '202310'+futuro_corregido_helen[futuro_corregido_helen['id'].isna()].index.astype(str)

In [27]:
amns['future'] = pd.concat([futuro_corregido_helen, amn_export[export_cols]]).drop_duplicates(subset=['Category', 'geometry'])

In [28]:
[amenities.to_parquet(f'/app/assets/amenities/{k}.parquet') for k, amenities in amns.items()];